In [155]:
import pandas as pd 
import matplotlib as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense
from tensorflow.keras.models import Model
from math import radians, sin, cos, sqrt, atan2

In [156]:
user = pd.read_csv('./data/user.csv')
destination = pd.read_csv('./data/destinasi-wisata-indonesia.csv')
bali =  pd.read_csv('./data/Dataset - tourismBali.csv')

In [157]:
user.head()

,User_Id,Location,Age
0,1,"Semarang, Jawa Tengah",20
1,2,"Bekasi, Jawa Barat",21
2,3,"Cirebon, Jawa Barat",23
3,4,"Bekasi, Jawa Barat",21
4,5,"Lampung, Sumatera Selatan",20


In [158]:
bali.head()

,Place_Id,Place_Name,Description,Weekend Holiday Price,Weekday Price,Category,City,Rating,Alamat,Coordinate,Lat,Long,Gambar
0,1,Agrowisata Satria,Agrowisata Satria menawarkan 'rasa' Bali pada ...,50000,50000,Agrowisata,Gianyar,3.0,"Manukaya, Kec. Tampaksiring, Kabupaten Gianya...","-8.402228546131536, 115.32296693687739",-8.402229,115.322967,V
1,2,Agung Bali,Dapatkan berbagai produk oleh-oleh khas Bali b...,0,0,Belanja,Badung,4.0,"Jln. Dewi Sri No.18XX, Kuta, Badung, Bali","-8.700234336021559, 115.176534407375",-8.700234,115.176534,V
2,3,Wisata Air Panas Toya Bungkah,Pemandian air panas di dekat gunung Batur (Pem...,70000,70000,Alam,Bangli,4.0,"Batur Tengah, Kec. Kintamani, Kabupaten Bangli...","-8.251298580809106, 115.39981444195475",-8.251299,115.399814,V
3,4,Air Terjun Aling-Aling,Keberadaan air terjun ini semakin mempercantik...,20000,20000,Alam,Buleleng,4.5,"Jl. Raya Desa Sambangan, Banjar, Sambangan, Ke...","-8.173073590748519, 115.10513024047802",-8.173074,115.105130,V
4,5,Air Terjun Tegenungan,Bosan dengan wisata alam pantai dan ingin meni...,20000,20000,Alam,Gianyar,4.0,"Jl. Ir. Sutami, Kemenuh, Kec. Sukawati, Kabupa...","-8.575191460385602, 115.28870189592831",-8.575191,115.288702,V


In [159]:
bali[['Weekday Price', 'Weekend Holiday Price']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   Weekday Price          75 non-null     int64
 1   Weekend Holiday Price  75 non-null     int64
dtypes: int64(2)
memory usage: 1.3 KB


In [160]:
mean = (bali['Weekday Price'].values + bali['Weekend Holiday Price'].values) / 2

In [161]:
mean

array([ 50000.,      0.,  70000.,  20000.,  20000.,  15000., 300000.,
       102500.,  20000., 500000.,      0.,  32500.,  50000.,      0.,
        15000.,  30000.,  10000.,  10000.,      0.,      0.,      0.,
        95000.,  30000.,  25000.,  15000.,  10000.,      0.,  40000.,
            0.,      0.,  25000.,      0.,  60000.,  25000.,  25000.,
        10000.,  10000.,      0.,      0.,  10000.,  10000.,      0.,
            0.,   5000.,      0.,   8000.,      0.,   5000.,   8000.,
            0.,      0.,  10000.,      0.,      0.,      0.,      0.,
        30000.,  30000.,  10000.,  50000.,  15000.,      0.,  15000.,
        15000.,  30000.,  50000.,      0.,      0., 100000., 150000.,
        10000.,  25000., 289000.,  10000.,      0.])

In [162]:
bali['Price'] = mean

In [163]:
bali.columns

Index(['Place_Id', 'Place_Name', 'Description', 'Weekend Holiday Price',
       'Weekday Price', 'Category', 'City', 'Rating', 'Alamat', 'Coordinate',
       'Lat', 'Long', 'Gambar', 'Price'],
      dtype='object')

In [164]:
destination.columns

Index(['Place_Id', 'Place_Name', 'Description', 'Category', 'City', 'Price',
       'Rating', 'Time_Minutes', 'Coordinate', 'Lat', 'Long', 'Column1', '_1',
       'Rating_Count'],
      dtype='object')

In [165]:
bali = bali[['Place_Id', 'Place_Name', 'Description', 'Category', 'City', 'Price', 'Rating', 'Coordinate', 'Lat', 'Long']]

In [166]:
bali['Category'] = bali['Category'].replace('Bali')

C:\Users\Akmal Fauzan\AppData\Local\Temp\ipykernel_32972\352904884.py:1: FutureWarning: Series.replace without 'value' and with non-dict-like 'to_replace' is deprecated and will raise in a future version. Explicitly specify the new values instead.
  bali['Category'] = bali['Category'].replace('Bali')


In [167]:
destination = pd.concat([destination, bali], ignore_index=True)

In [168]:
destination.shape

(512, 14)

In [169]:
destination.head()

,Place_Id,Place_Name,Description,Category,City,Price,Rating,Time_Minutes,Coordinate,Lat,Long,Column1,_1,Rating_Count
0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,20000.0,46.0,15.0,"{'lat': -6.1753924, 'lng': 106.8271528}",-61753924.0,1.068272e+09,NaN,1.0,18.0
1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,0.0,46.0,90.0,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-61376448.0,1.068171e+09,NaN,2.0,25.0
2,3,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,Jakarta,270000.0,46.0,360.0,"{'lat': -6.125312399999999, 'lng': 106.8335377}",-61253124.0,1.068335e+09,NaN,3.0,19.0
3,4,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Taman Hiburan,Jakarta,10000.0,45.0,NaN,"{'lat': -6.302445899999999, 'lng': 106.8951559}",-63024459.0,1.068952e+09,NaN,4.0,21.0
4,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Taman Hiburan,Jakarta,94000.0,45.0,60.0,"{'lat': -6.12419, 'lng': 106.839134}",-612419.0,1.068391e+08,NaN,5.0,24.0


In [170]:
destination['City'].value_counts()

City
Yogyakarta    126
Bandung       124
Jakarta        84
Semarang       57
Surabaya       46
Gianyar        21
Badung         20
Bangli          7
Klungkung       7
Karangasem      7
Buleleng        5
Denpasar        4
Tabanan         4
Name: count, dtype: int64

In [171]:
destination['Category'].unique()

array(['Budaya', 'Taman Hiburan', 'Cagar Alam', 'Bahari',
       'Pusat Perbelanjaan', 'Tempat Ibadah', 'Agrowisata', 'Belanja',
       'Alam', 'Rekreasi', 'Religius', 'Pantai'], dtype=object)

In [172]:
destination['Category'] = destination['Category'].replace('Pantai', 'Bahari')

In [173]:
destination[destination['Category'] == 'Bahari' ].sample(7)

,Place_Id,Place_Name,Description,Category,City,Price,Rating,Time_Minutes,Coordinate,Lat,Long,Column1,_1,Rating_Count
45,46,Pulau Semak Daun,Pulau Semak Daun merupakan salah satu pulau ya...,Bahari,Jakarta,40000.0,4.0,NaN,"{'lat': -5.729671599999999, 'lng': 106.5714158}",-5.729672e+07,1.065714e+09,NaN,46.0,23.0
198,199,Pantai Sedahan,Pantai Sedahan ini memiliki garis pantai yang ...,Bahari,Yogyakarta,5000.0,45.0,NaN,"{'lat': -8.1978943, 'lng': 110.7208535}",-8.197894e+07,1.107209e+09,NaN,199.0,19.0
483,47,Pantai Nusa Dua,Pantai Nusa Dua terkenal sebagai pantai terber...,Bahari,Badung,0.0,4.0,NaN,"-8.795533863779477, 115.23284296008501",-8.795534e+00,1.152328e+02,NaN,NaN,NaN
201,202,Pantai Ngandong,"Pantai Ngandong terletak di Desa Sidoarjo, Kec...",Bahari,Yogyakarta,10000.0,45.0,NaN,"{'lat': -8.146423, 'lng': 110.6061408}",-8.146423e+06,1.106061e+09,NaN,202.0,28.0
13,14,Pulau Pelangi,Pulau Pelangi adalah sebuah pulau yang terleta...,Bahari,Jakarta,900000.0,48.0,NaN,"{'lat': -5.587055, 'lng': 106.5885}",-5.587055e+06,1.065885e+06,NaN,14.0,18.0
190,191,Pantai Drini,Pantai Drini adalah salah satu objek wisata ya...,Bahari,Yogyakarta,10000.0,45.0,NaN,"{'lat': -8.1384509, 'lng': 110.577522}",-8.138451e+07,1.105775e+08,NaN,191.0,22.0
197,198,Pantai Greweng,"Di Kabupaten Gunungkidul, tidak sulit memilih ...",Bahari,Yogyakarta,5000.0,46.0,NaN,"{'lat': -8.197864399999998, 'lng': 110.7186216}",-8.197864e+07,1.107186e+09,NaN,198.0,19.0


In [174]:
destination.isnull().sum()

Place_Id          0
Place_Name        0
Description       0
Category          0
City              0
Price             0
Rating            0
Time_Minutes    307
Coordinate        0
Lat               0
Long              0
Column1         512
_1               75
Rating_Count     75
dtype: int64

In [175]:
destination.drop(columns=['Place_Id','Time_Minutes', 'Column1', '_1', 'Rating_Count'], inplace=True)

In [176]:
destination.columns

Index(['Place_Name', 'Description', 'Category', 'City', 'Price', 'Rating',
       'Coordinate', 'Lat', 'Long'],
      dtype='object')

In [177]:
destination.duplicated().sum()

0

In [178]:
destination.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512 entries, 0 to 511
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Place_Name   512 non-null    object 
 1   Description  512 non-null    object 
 2   Category     512 non-null    object 
 3   City         512 non-null    object 
 4   Price        512 non-null    float64
 5   Rating       512 non-null    float64
 6   Coordinate   512 non-null    object 
 7   Lat          512 non-null    float64
 8   Long         512 non-null    float64
dtypes: float64(4), object(5)
memory usage: 36.1+ KB


##REKOMENDASI BY KATEGORI WISATA

### PREPROCESSING
di encode + cari kesamaan pake cosine similarity

In [179]:
oneHotCategory = OneHotEncoder(sparse=False)
oneHotCity = OneHotEncoder(sparse=False)

In [180]:
categoryEncoded = oneHotCategory.fit_transform(destination[['Category']])
cityEncoded = oneHotCity.fit_transform(destination[['City']])

c:\Users\Akmal Fauzan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\Akmal Fauzan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [181]:
categoryEncoded = pd.DataFrame(categoryEncoded, columns=oneHotCategory.get_feature_names_out(['Category']))
cityEncoded = pd.DataFrame(cityEncoded, columns=oneHotCity.get_feature_names_out(['City']))

In [182]:
categoryEncoded

,Category_Agrowisata,Category_Alam,Category_Bahari,Category_Belanja,Category_Budaya,Category_Cagar Alam,Category_Pusat Perbelanjaan,Category_Rekreasi,Category_Religius,Category_Taman Hiburan,Category_Tempat Ibadah
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
507,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
509,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
510,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [183]:
cityEncoded

,City_Badung,City_Bandung,City_Bangli,City_Buleleng,City_Denpasar,City_Gianyar,City_Jakarta,City_Karangasem,City_Klungkung,City_Semarang,City_Surabaya,City_Tabanan,City_Yogyakarta
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
509,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
510,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [184]:
cosineSim = cosine_similarity(categoryEncoded)
cosineCity = cosine_similarity(cityEncoded)

In [185]:
index_combined = destination['Place_Name'] + " - " + destination['City']
cosineSim = pd.DataFrame(cosineSim, index=index_combined, columns=index_combined)
cosineCity = pd.DataFrame(cosineCity, index=index_combined, columns=index_combined)

In [186]:
cosineSim

,Monumen Nasional - Jakarta,Kota Tua - Jakarta,Dunia Fantasi - Jakarta,Taman Mini Indonesia Indah (TMII) - Jakarta,Atlantis Water Adventure - Jakarta,Taman Impian Jaya Ancol - Jakarta,Kebun Binatang Ragunan - Jakarta,Ocean Ecopark - Jakarta,Pelabuhan Marina - Jakarta,Pulau Tidung - Jakarta,Pulau Bidadari - Jakarta,Pulau Pari - Jakarta,Pulau Pramuka - Jakarta,Pulau Pelangi - Jakarta,Pasar Seni - Jakarta,Jembatan Kota Intan - Jakarta,Museum Fatahillah - Jakarta,Museum Bank Indonesia - Jakarta,Kidzania - Jakarta,Museum Taman Prasasti - Jakarta,Museum Wayang - Jakarta,Masjid Istiqlal - Jakarta,Gereja Katedral - Jakarta,Museum Nasional - Jakarta,Pasar Tanah Abang - Jakarta,Pecinan Glodok - Jakarta,Sea World - Jakarta,Wisata Agro Edukatif Istana Susu Cibugary - Jakarta,Wisata Kuliner Pecenongan - Jakarta,Taman Menteng - Jakarta,Wisata Alam Mangrove Angke - Jakarta,Setu Babakan - Jakarta,Taman Suropati - Jakarta,Pasar Taman Puring - Jakarta,Grand Indonesia Mall - Jakarta,Skyrink - Mall Taman Anggrek - Jakarta,Bumi Perkemahan Cibubur - Jakarta,Istana Negara Republik Indonesia - Jakarta,Museum Macan (Modern and Contemporary Art in Nusantara) - Jakarta,Galeri Nasional Indonesia - Jakarta,Museum Bahari Jakarta - Jakarta,Museum Seni Rupa dan Kramik - Jakarta,The Escape Hunt - Jakarta,Monumen Selamat Datang - Jakarta,Jakarta Aquarium dan Safari - Jakarta,Pulau Semak Daun - Jakarta,Taman Situ Lembang - Jakarta,Taman Ismail Marzuki - Jakarta,Galeri Indonesia Kaya - Jakarta,Museum Joang 45 - Jakarta,Jakarta Planetarium - Jakarta,Kampung Cina - Jakarta,Rumah Sipitung - Jakarta,Taman Agrowisata Cilangkap - Jakarta,Klenteng Jin De Yuan - Jakarta,Pantai Ancol - Jakarta,Taman Lapangan Banteng - Jakarta,Taman Ayodya - Jakarta,Museum Sumpah Pemuda - Jakarta,Museum Tekstil - Jakarta,Tugu Proklamasi - Jakarta,Taman Legenda Keong Emas - Jakarta,Museum Kebangkitan Nasional - Jakarta,Museum Sasmita Loka Ahmad Yani - Jakarta,Museum Basoeki Abdullah - Jakarta,Museum Layang-layang - Jakarta,Margasatwa Muara Angke - Jakarta,Pasar Petak Sembilan - Jakarta,Freedom Library - Jakarta,Perpustakaan Nasional - Jakarta,Cibubur Garden Diary (Cibugary) - Jakarta,Waterboom PIK (Pantai Indah Kapuk) - Jakarta,Museum Perangko - Jakarta,Museum Tengah Kebun - Jakarta,SnowBay Waterpark - Jakarta,Taman Cattleya - Jakarta,Taman Hutan Tebet - Jakarta,Hutan Kota Srengseng - Jakarta,Taman Spathodea - Jakarta,Plaza Indonesia - Jakarta,Mall Thamrin City - Jakarta,Museum Satria Mandala - Jakarta,Alive Museum Ancol - Jakarta,Kawasan Kuliner BSM - Jakarta,Taman Pintar Yogyakarta - Yogyakarta,Keraton Yogyakarta - Yogyakarta,Sindu Kusuma Edupark (SKE) - Yogyakarta,Museum Benteng Vredeburg Yogyakarta - Yogyakarta,De Mata Museum Jogja - Yogyakarta,Kampung Wisata Taman Sari - Yogyakarta,Situs Warungboto - Yogyakarta,Nol Kilometer Jl.Malioboro - Yogyakarta,Gembira Loka Zoo - Yogyakarta,Sumur Gumuling - Yogyakarta,Desa Wisata Sungai Code Jogja Kota - Yogyakarta,Alun Alun Selatan Yogyakarta - Yogyakarta,Monumen Yogya Kembali - Yogyakarta,Taman Pelangi Yogyakarta - Yogyakarta,Kampung Wisata Kadipaten - Yogyakarta,Taman Budaya Yogyakarta - Yogyakarta,Kampung Wisata Sosro Menduran - Yogyakarta,Monumen Batik Yogyakarta - Yogyakarta,Tugu Pal Putih Jogja - Yogyakarta,Tebing Breksi - Yogyakarta,Candi Donotirto - Yogyakarta,Kawasan Malioboro - Yogyakarta,Bangsal Pagelaran - Yogyakarta,Embung Tambakboyo - Yogyakarta,Hutan Pinus Pengger - Yogyakarta,Monumen Serangan Umum 1 Maret - Yogyakarta,Puncak Pinus Becici - Yogyakarta,Bukit Bintang Yogyakarta - Yogyakarta,Gedung Agung Yogyakarta - Yogyakarta,The Lost World Castle - Yogyakarta,Monumen Sanapati - Yogyakarta,Jurang Tembelan Kanigoro - Yogyakarta,The World Landmarks - Merapi Park Yogyakarta - Yogyakarta,Museum Sonobudoyo Unit I - Yogyakarta,Lava Bantal - Yogyakarta,Bukit Panguk Kediwung - Yogyakarta,Candi Prambanan - Yogyakarta,Watu Goyang - Yogyakarta,Kampung Wisata Rejowinangun - Yogyakarta,Kauman Pakualaman Yogyakarta - Yogyakarta,Alun-alun Utara 

In [187]:
cosineCity

,Monumen Nasional - Jakarta,Kota Tua - Jakarta,Dunia Fantasi - Jakarta,Taman Mini Indonesia Indah (TMII) - Jakarta,Atlantis Water Adventure - Jakarta,Taman Impian Jaya Ancol - Jakarta,Kebun Binatang Ragunan - Jakarta,Ocean Ecopark - Jakarta,Pelabuhan Marina - Jakarta,Pulau Tidung - Jakarta,Pulau Bidadari - Jakarta,Pulau Pari - Jakarta,Pulau Pramuka - Jakarta,Pulau Pelangi - Jakarta,Pasar Seni - Jakarta,Jembatan Kota Intan - Jakarta,Museum Fatahillah - Jakarta,Museum Bank Indonesia - Jakarta,Kidzania - Jakarta,Museum Taman Prasasti - Jakarta,Museum Wayang - Jakarta,Masjid Istiqlal - Jakarta,Gereja Katedral - Jakarta,Museum Nasional - Jakarta,Pasar Tanah Abang - Jakarta,Pecinan Glodok - Jakarta,Sea World - Jakarta,Wisata Agro Edukatif Istana Susu Cibugary - Jakarta,Wisata Kuliner Pecenongan - Jakarta,Taman Menteng - Jakarta,Wisata Alam Mangrove Angke - Jakarta,Setu Babakan - Jakarta,Taman Suropati - Jakarta,Pasar Taman Puring - Jakarta,Grand Indonesia Mall - Jakarta,Skyrink - Mall Taman Anggrek - Jakarta,Bumi Perkemahan Cibubur - Jakarta,Istana Negara Republik Indonesia - Jakarta,Museum Macan (Modern and Contemporary Art in Nusantara) - Jakarta,Galeri Nasional Indonesia - Jakarta,Museum Bahari Jakarta - Jakarta,Museum Seni Rupa dan Kramik - Jakarta,The Escape Hunt - Jakarta,Monumen Selamat Datang - Jakarta,Jakarta Aquarium dan Safari - Jakarta,Pulau Semak Daun - Jakarta,Taman Situ Lembang - Jakarta,Taman Ismail Marzuki - Jakarta,Galeri Indonesia Kaya - Jakarta,Museum Joang 45 - Jakarta,Jakarta Planetarium - Jakarta,Kampung Cina - Jakarta,Rumah Sipitung - Jakarta,Taman Agrowisata Cilangkap - Jakarta,Klenteng Jin De Yuan - Jakarta,Pantai Ancol - Jakarta,Taman Lapangan Banteng - Jakarta,Taman Ayodya - Jakarta,Museum Sumpah Pemuda - Jakarta,Museum Tekstil - Jakarta,Tugu Proklamasi - Jakarta,Taman Legenda Keong Emas - Jakarta,Museum Kebangkitan Nasional - Jakarta,Museum Sasmita Loka Ahmad Yani - Jakarta,Museum Basoeki Abdullah - Jakarta,Museum Layang-layang - Jakarta,Margasatwa Muara Angke - Jakarta,Pasar Petak Sembilan - Jakarta,Freedom Library - Jakarta,Perpustakaan Nasional - Jakarta,Cibubur Garden Diary (Cibugary) - Jakarta,Waterboom PIK (Pantai Indah Kapuk) - Jakarta,Museum Perangko - Jakarta,Museum Tengah Kebun - Jakarta,SnowBay Waterpark - Jakarta,Taman Cattleya - Jakarta,Taman Hutan Tebet - Jakarta,Hutan Kota Srengseng - Jakarta,Taman Spathodea - Jakarta,Plaza Indonesia - Jakarta,Mall Thamrin City - Jakarta,Museum Satria Mandala - Jakarta,Alive Museum Ancol - Jakarta,Kawasan Kuliner BSM - Jakarta,Taman Pintar Yogyakarta - Yogyakarta,Keraton Yogyakarta - Yogyakarta,Sindu Kusuma Edupark (SKE) - Yogyakarta,Museum Benteng Vredeburg Yogyakarta - Yogyakarta,De Mata Museum Jogja - Yogyakarta,Kampung Wisata Taman Sari - Yogyakarta,Situs Warungboto - Yogyakarta,Nol Kilometer Jl.Malioboro - Yogyakarta,Gembira Loka Zoo - Yogyakarta,Sumur Gumuling - Yogyakarta,Desa Wisata Sungai Code Jogja Kota - Yogyakarta,Alun Alun Selatan Yogyakarta - Yogyakarta,Monumen Yogya Kembali - Yogyakarta,Taman Pelangi Yogyakarta - Yogyakarta,Kampung Wisata Kadipaten - Yogyakarta,Taman Budaya Yogyakarta - Yogyakarta,Kampung Wisata Sosro Menduran - Yogyakarta,Monumen Batik Yogyakarta - Yogyakarta,Tugu Pal Putih Jogja - Yogyakarta,Tebing Breksi - Yogyakarta,Candi Donotirto - Yogyakarta,Kawasan Malioboro - Yogyakarta,Bangsal Pagelaran - Yogyakarta,Embung Tambakboyo - Yogyakarta,Hutan Pinus Pengger - Yogyakarta,Monumen Serangan Umum 1 Maret - Yogyakarta,Puncak Pinus Becici - Yogyakarta,Bukit Bintang Yogyakarta - Yogyakarta,Gedung Agung Yogyakarta - Yogyakarta,The Lost World Castle - Yogyakarta,Monumen Sanapati - Yogyakarta,Jurang Tembelan Kanigoro - Yogyakarta,The World Landmarks - Merapi Park Yogyakarta - Yogyakarta,Museum Sonobudoyo Unit I - Yogyakarta,Lava Bantal - Yogyakarta,Bukit Panguk Kediwung - Yogyakarta,Candi Prambanan - Yogyakarta,Watu Goyang - Yogyakarta,Kampung Wisata Rejowinangun - Yogyakarta,Kauman Pakualaman Yogyakarta - Yogyakarta,Alun-alun Utara 

### Content-Based

In [188]:
def getRecommendationsByCategory(category, destination, similarity, top_n=5):
    relevant_places = destination[destination['Category'] == category]

    relevant_indices = relevant_places['Place_Name'] + " - " + relevant_places['City']

    if relevant_indices.empty:
        return "Tidak ada tempat wisata dengan kategori yang diminta."

    recommendations = {}
    for index in relevant_indices:
        similarityScores = similarity[index]

        sorted_scores = similarityScores.sort_values(ascending=False).iloc[1:top_n+1]
        recommendations[index] = sorted_scores

    return recommendations

In [189]:
recommendationsByCategory = getRecommendationsByCategory("Monumen", destination, cosineSim, top_n=5)
print("Rekomendasi Berdasarkan Kategori Wisata:")
print(recommendationsByCategory)

Rekomendasi Berdasarkan Kategori Wisata:
Tidak ada tempat wisata dengan kategori yang diminta.


In [190]:
def getRecommendationsByCity(city, destination, similarity, top_n=5):
    relevant_places = destination[destination['City'] == city]

    relevant_indices = relevant_places['Place_Name'] + " - " + relevant_places['City']

    if relevant_indices.empty:
        return "Tidak ada tempat wisata di kota yang diminta."

    recommendations = {}
    for index in relevant_indices:
        similarityScores = similarity[index]
        
        sorted_scores = similarityScores.sort_values(ascending=False).iloc[1:top_n+1]
        recommendations[index] = sorted_scores

    return recommendations

In [191]:
recommendationsByCity = getRecommendationsByCity("Jakarta", destination, cosineCity, top_n=5)
print("\nRekomendasi Berdasarkan Kota:")
print(recommendationsByCity)


Rekomendasi Berdasarkan Kota:
{'Monumen Nasional - Jakarta': Klenteng Jin De Yuan - Jakarta           1.0
Museum Kebangkitan Nasional - Jakarta    1.0
Taman Legenda Keong Emas - Jakarta       1.0
Tugu Proklamasi - Jakarta                1.0
Museum Tekstil - Jakarta                 1.0
Name: Monumen Nasional - Jakarta, dtype: float64, 'Kota Tua - Jakarta': Klenteng Jin De Yuan - Jakarta           1.0
Museum Kebangkitan Nasional - Jakarta    1.0
Taman Legenda Keong Emas - Jakarta       1.0
Tugu Proklamasi - Jakarta                1.0
Museum Tekstil - Jakarta                 1.0
Name: Kota Tua - Jakarta, dtype: float64, 'Dunia Fantasi - Jakarta': Klenteng Jin De Yuan - Jakarta           1.0
Museum Kebangkitan Nasional - Jakarta    1.0
Taman Legenda Keong Emas - Jakarta       1.0
Tugu Proklamasi - Jakarta                1.0
Museum Tekstil - Jakarta                 1.0
Name: Dunia Fantasi - Jakarta, dtype: float64, 'Taman Mini Indonesia Indah (TMII) - Jakarta': Klenteng Jin De Yuan - Jakar

In [192]:
weightCategory = 0.5  
weightCity = 0.5

combinedSimilarity = (weightCategory * cosineSim) + (weightCity * cosineCity)

In [193]:
def getCombinedRecommendations(category, city, destination, combined_similarity, top_n=5):
    relevant_places = destination[(destination['City'] == city) & 
                                     (destination['Category'] == category)]
    
    relevant_indices = relevant_places['Place_Name'] + " - " + relevant_places['City']
    
    if relevant_indices.empty:
        return "Tidak ada tempat yang cocok dengan kategori dan kota yang diminta."

    recommendations = {}
    for index in relevant_indices:
        similarityScores = combined_similarity[index]
        
        sorted_scores = similarityScores.sort_values(ascending=False).iloc[1:top_n+1]
        recommendations[index] = sorted_scores
    
    return recommendations


In [194]:
budaya = destination[(destination['Category'] == 'Budaya') & (destination['City'] == 'Yogyakarta')]

In [195]:
budaya

,Place_Name,Description,Category,City,Price,Rating,Coordinate,Lat,Long
85,Keraton Yogyakarta,Keraton Ngayogyakarta Hadiningrat atau Keraton...,Budaya,Yogyakarta,15000.0,46.0,"{'lat': -7.8052845, 'lng': 110.3642031}",-78052845.0,1.103642e+09
87,Museum Benteng Vredeburg Yogyakarta,Museum Benteng Vredeburg (bahasa Jawa: ꦩꦸꦱꦶꦪꦸꦩ...,Budaya,Yogyakarta,3000.0,46.0,"{'lat': -7.800201599999999, 'lng': 110.3663044}",-78002016.0,1.103663e+09
88,De Mata Museum Jogja,Museum De Mata merupakan salah satu museum yan...,Budaya,Yogyakarta,50000.0,44.0,"{'lat': -7.816315599999999, 'lng': 110.3871442}",-78163156.0,1.103871e+09
96,Monumen Yogya Kembali,Museum Monumen Yogya Kembali (bahasa Jawa: ꦩꦺꦴ...,Budaya,Yogyakarta,15000.0,45.0,"{'lat': -7.7495904, 'lng': 110.3696068}",-77495904.0,1.103696e+09
98,Kampung Wisata Kadipaten,Kampung Wisata Kadipaten secaara kewilayahan b...,Budaya,Yogyakarta,0.0,44.0,"{'lat': -7.806093000000001, 'lng': 110.35831}",-7806093.0,1.103583e+07
99,Taman Budaya Yogyakarta,Taman Budaya Yogyakarta (TBY) (Hanacaraka:ꦠꦩꦤ꧀...,Budaya,Yogyakarta,0.0,45.0,"{'lat': -7.8001041, 'lng': 110.3676579}",-78001041.0,1.103677e+09
100,Kampung Wisata Sosro Menduran,Kampung wisata Sosromenduran merupakan kampung...,Budaya,Yogyakarta,0.0,4.0,"{'lat': -7.792189999999999, 'lng': 110.362151}",-779219.0,1.103622e+08
101,Monumen Batik Yogyakarta,Perhatian pemerintah megenai kebudayan batik i...,Budaya,Yogyakarta,40000.0,45.0,"{'lat': -7.8011159, 'lng': 110.3646172}",-78011159.0,1.103646e+09
103,Tebing Breksi,Tebing Breksi merupakan tempat wisata yang ber...,Budaya,Yogyakarta,20000.0,44.0,"{'lat': -7.781571400000001, 'lng': 110.5045827}",-77815714.0,1.105046e+09
104,Candi Donotirto,"Walaupun melekat dengan kata candi, Candi Dono...",Budaya,Yogyakarta,0.0,44.0,"{'lat': -7.795561299999998, 'lng': 110.3612095}",-77955613.0,1.103612e+09


In [196]:
# Buat rekomendasi berdasarkan kategori wisata dan kota
recommendations_combined = getCombinedRecommendations("Budaya", "Yogyakarta", destination, combinedSimilarity, top_n=5)
print("Rekomendasi Berdasarkan Kategori dan Kota:")
print(recommendations_combined)


Rekomendasi Berdasarkan Kategori dan Kota:
{'Keraton Yogyakarta - Yogyakarta': Bangsal Pagelaran - Yogyakarta              1.0
Kampung Wisata Rejowinangun - Yogyakarta    1.0
Watu Goyang - Yogyakarta                    1.0
Candi Prambanan - Yogyakarta                1.0
Bukit Panguk Kediwung - Yogyakarta          1.0
Name: Keraton Yogyakarta - Yogyakarta, dtype: float64, 'Museum Benteng Vredeburg Yogyakarta - Yogyakarta': Bangsal Pagelaran - Yogyakarta              1.0
Kampung Wisata Rejowinangun - Yogyakarta    1.0
Watu Goyang - Yogyakarta                    1.0
Candi Prambanan - Yogyakarta                1.0
Bukit Panguk Kediwung - Yogyakarta          1.0
Name: Museum Benteng Vredeburg Yogyakarta - Yogyakarta, dtype: float64, 'De Mata Museum Jogja - Yogyakarta': Bangsal Pagelaran - Yogyakarta              1.0
Kampung Wisata Rejowinangun - Yogyakarta    1.0
Watu Goyang - Yogyakarta                    1.0
Candi Prambanan - Yogyakarta                1.0
Bukit Panguk Kediwung - Yogyakart

### Rekomendasi Terdekat

In [197]:
destination['Lat'] = destination['Lat'].apply(lambda x: float(str(x).replace("'", "").replace("{", "").replace("}", "").split(':')[-1].strip()))
destination['Long'] = destination['Long'].apply(lambda x: float(str(x).replace("'", "").replace("{", "").replace("}", "").split(':')[-1].strip()))

In [198]:
destination[['Lat', 'Long']].sample(5)

,Lat,Long
378,-7.272162e+07,1.104698e+09
169,-7.607874e+07,1.102038e+09
472,-8.724677e+00,1.154476e+02
486,-8.695787e+00,1.154520e+02
292,-6.913275e+07,1.076095e+09


In [199]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

In [200]:
def get_nearest_places(user_lat, user_long, data, top_n=5):
    data['Distance_km'] = data.apply(lambda row: haversine(user_lat, user_long, row['Lat'], row['Long']), axis=1)
    nearest_places = data.sort_values(by='Distance_km').head(top_n)
    return nearest_places

In [201]:
user_lat, user_long = -8.172141e+00	, 1.149923e+02

In [202]:
nearest_places = get_nearest_places(user_lat, user_long, destination)

In [203]:
nearest_places[['Place_Name', 'Category', 'City', 'Distance_km']]

,Place_Name,Category,City,Distance_km
468,Krisna Oleh Oleh,Belanja,Buleleng,0.001663
440,Air Terjun Aling-Aling,Alam,Buleleng,12.419164
449,Bukit Wanagiri,Alam,Buleleng,14.781757
453,Danau Tamblingan,Alam,Buleleng,14.820360
445,Handara Iconic Gate,Budaya,Buleleng,20.350085


### Collaborative Filtering

In [204]:
num_users = 1000  
num_places = len(destination)  
embedding_dim = 50

In [205]:
user_input = Input(shape=(1,), name='user_input')
place_input = Input(shape=(1,), name='place_input')

In [206]:
user_embedding = Embedding(input_dim=num_users, output_dim=embedding_dim, name='user_embedding')(user_input)
place_embedding = Embedding(input_dim=num_places, output_dim=embedding_dim, name='place_embedding')(place_input)

In [207]:
user_vec = Flatten()(user_embedding)
place_vec = Flatten()(place_embedding)

In [208]:
similarityScores = Dot(axes=1)([user_vec, place_vec])

In [209]:
model = Model(inputs=[user_input, place_input], outputs=similarityScores)
model.compile(optimizer='adam', loss='mse')

In [210]:
user_ids = np.array([1, 2, 3, 4])  
place_ids = np.array([10, 20, 30, 40])  
ratings = np.array([4.0, 5.0, 3.0, 4.5])

In [211]:
model.fit([user_ids, place_ids], ratings, epochs=10, verbose=1)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 556ms/step - loss: 17.5302
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 17.5103
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 17.4903
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 17.4703
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 17.4502
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 17.4300
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 17.4096
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 17.3891
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 17.3683
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 17.3472


In [212]:
user_id = np.array([1])  
place_id = np.array([10])

In [213]:
predicted_score = model.predict([user_id, place_id])
print(f"skor prediksi user {user_id[0]} dan tempat {place_id[0]}: {predicted_score[0][0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
skor prediksi user 1 dan tempat 10: 0.031093204393982887
